In [1]:
import json
import os

In [2]:
def ParsingJson(textname, filename):
    with open("./data/raw/sample/"+textname+"/"+filename+".json", "r", encoding="utf8") as f:
        contents = f.read()
        file = json.loads(contents)
    keyJson = []
    if textname=="chat":
        keyJson = "utterances"
    elif textname=="news":
        keyJson = "sentences"
    
    countSentences = len(file[keyJson])
    
    result_list = []
    
    for k in range(countSentences):
        sentence = file[keyJson][k]["text"]

        len_timex3 = len(file[keyJson][k]["timex3"])
        len_event = len(file[keyJson][k]["event"])
        info_list = [[] for i in range(len_event+len_timex3)]

        for i in range(len_event):
            event_s, event_e = file[keyJson][k]["event"][i]["extent"]
            info_list[i].append(event_s)
            info_list[i].append(event_e)
            info_list[i].append(file[keyJson][k]["event"][i]["class"])

        for i in range(len_timex3):
            timex3_s, timex3_e = file[keyJson][k]["timex3"][i]["extent"]
            info_list[i+len_event].append(timex3_s)
            info_list[i+len_event].append(timex3_e)
            info_list[i+len_event].append(file[keyJson][k]["timex3"][i]["type"])
            

        info_list.sort()
        print(info_list)

        named_entity = [0 for i in range(len(sentence))]
        for i in range(len(info_list)):
            start = info_list[i][0]
            end = info_list[i][1]
            netype = info_list[i][2]
            BorI = True
            for j in range(start, end):
                if BorI:
                    named_entity[j] = (netype, '-B')
                    BorI = False
                else:
                    named_entity[j] = (netype, '-I')

        label = []
        for i in range(len(sentence)):
            if named_entity[i]:
                label.append(named_entity[i][0]+named_entity[i][1])
            else:
                label.append('O')
            
        result = sentence + '\t'
        for i in range(len(label)):
            result += label[i] + ' '
        result = result.strip()
        result += '\t'
        
        print(result)
        
        tlinkList = file["tlink"]
        for i in range(len(tlinkList)):
            timex3NE = tlinkList[i]["arg1"]
            eventNE = tlinkList[i]["arg2"]
            sen_index = int(timex3NE.split('_')[1][1:])
            if (len_timex3 >= sen_index):
                if (len_event >= sen_index):
                    tne_index = int(timex3NE.split('_')[2])
                    ene_index = int(eventNE.split('_')[2])
                    print(tne_index, type(tne_index), ene_index, type(ene_index))

                    if (file[keyJson][k]["timex3"][tne_index]["id"] == timex3NE):
                        if (file[keyJson][k]["event"][ene_index]["id"] == eventNE):
                            time_link = file[keyJson][k]["timex3"][tne_index]["text"]
                            event_link = file[keyJson][k]["event"][ene_index]["text"]
                            tlink_type = tlinkList[i]["relType"]
                            res_tlink = time_link+" "+event_link+" "+tlink_type
                            result = result + res_tlink
                            print(result)
                            
        result_list.append(result)
        # 예상 result_list[i]의 모습
        # 문장 \t BIO tag \t 시간표현 \s 사건표현 \s 관계
    return result_list

In [3]:
def saveTxt(textname, filename, res):
    f = open("./data/tlink/"+textname+"/"+filename, "w")
    for i in range(len(res)):
        f.write(res[i]+'\n')
    f.close()

In [4]:
# 파일이름 받아오기

text_name = ["chat", "news"]
file_list = []
for i in range(len(text_name)):
    file_list.append(os.listdir("./data/raw/sample/"+text_name[i]))

In [5]:
for i in range(len(file_list)):
    for j in range(len(file_list[i])):
        listFilename = file_list[i][j].split('.')[:-1]
        file_list[i][j] = listFilename[0] + '.' + listFilename[1]

In [6]:
# json파일에서 데이터 추출하기

for i in range(len(file_list)):
    for j in range(len(file_list[i])):
        result = ParsingJson(text_name[i], file_list[i][j])
        saveTxt(text_name[i], file_list[i][j], result)

[[15, 18, 'OCCURRENCE'], [27, 28, 'OCCURRENCE']]
데이팅 앱을 쓰는 사람들이 늘어나는데 어떻게 생각하세요?	O O O O O O O O O O O O O O O OCCURRENCE-B OCCURRENCE-I OCCURRENCE-I O O O O O O O O O OCCURRENCE-B O O O	
[[18, 21, 'OCCURRENCE'], [29, 32, 'OCCURRENCE'], [31, 32, 'OCCURRENCE'], [34, 35, 'I_STATE']]
사람을 만나는 수단 중 하나라고 생각해서 나쁘다고는 생각하지 않아요.	O O O O O O O O O O O O O O O O O O OCCURRENCE-B OCCURRENCE-I OCCURRENCE-I O O O O O O O O OCCURRENCE-B OCCURRENCE-I OCCURRENCE-B O O I_STATE-B O O O	
[[18, 21, 'OCCURRENCE'], [23, 25, 'DATE'], [32, 33, 'OCCURRENCE'], [38, 39, 'OCCURRENCE']]
제 지인 중에서도 앱으로 연애를 시작해서 작년에 결혼까지 한 사람이 있어요.	O O O O O O O O O O O O O O O O O O OCCURRENCE-B OCCURRENCE-I OCCURRENCE-I O O DATE-B DATE-I O O O O O O O OCCURRENCE-B O O O O O OCCURRENCE-B O O O	
[[8, 10, 'DATE'], [13, 14, 'OCCURRENCE'], [26, 27, 'OCCURRENCE']]
저도 궁금해서 어제 깔아봤는데 아직은 거부감이 있더라고요,	O O O O O O O O DATE-B DATE-I O O O OCCURRENCE-B O O O O O O O O O O O O OCCURRENCE-B O O O O O	
[[12, 13, 'OCCURRENCE'], [25, 28, 'OCCURRENCE'],

TypeError: can only concatenate list (not "str") to list

In [ ]:
print(result)